# Imports

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

In [54]:
# from sklearn import datasets
from sklearn.datasets import load_digits
from sklearn.metrics import classification_report, confusion_matrix

# Import data

In [55]:
TRAIN_FILE ='train-images.idx3-ubyte'
TRAIN_LABEL ='train-labels.idx1-ubyte'
TEST_FILE ='t10k-images.idx3-ubyte'
TEST_LABEL ='t10k-labels.idx1-ubyte'

In [56]:
with open(TRAIN_FILE, 'rb') as ftemp:
    datatemp = np.fromfile(ftemp, dtype = np.ubyte)
    X_train = datatemp[16::].reshape(60000,784)
    print('Size of the training set:',X_train.shape)

with open(TRAIN_LABEL, 'rb') as ftemp:
    datatemp = np.fromfile(ftemp, dtype = np.ubyte)
    y_train = datatemp[8::]
    print('Size of the training labels:',y_train.shape)
    
with open(TEST_FILE, 'rb') as ftemp:
    datatemp = np.fromfile(ftemp, dtype = np.ubyte)
    X_test = datatemp[16::].reshape(10000,784)
    print('Size of the test set:',X_test.shape)
    
with open(TEST_LABEL, 'rb') as ftemp:
    datatemp = np.fromfile(ftemp, dtype = np.ubyte)
    y_test = datatemp[8::]
    print('Size of the test label:',y_test.shape)

Size of the training set: (60000, 784)
Size of the training labels: (60000,)
Size of the test set: (10000, 784)
Size of the test label: (10000,)


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Initial training

In [59]:
classifier = SVC(gamma='auto', kernel='linear')
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [60]:
predictions = classifier.predict(X_test)

In [61]:
print(confusion_matrix(y_test, predictions))

[[ 957    0    6    2    0    8    3    1    2    1]
 [   0 1118    2    2    0    1    2    2    8    0]
 [   9   10  939   29   11    6    7    4   15    2]
 [   4    2   27  923    2   25    2    9   15    1]
 [   0    2   12    0  924    2    3    1    4   34]
 [  12    3    8   54    5  767   13    2   24    4]
 [  16    3   18    1    9   15  894    1    1    0]
 [   2   13   26   10   15    1    0  926    3   32]
 [  12    9   22   41   12   38    7    6  822    5]
 [   8    8    5   11   42    7    0   34   11  883]]


In [62]:
print('\n')
print(classification_report(y_test, predictions))



              precision    recall  f1-score   support

           0       0.94      0.98      0.96       980
           1       0.96      0.99      0.97      1135
           2       0.88      0.91      0.90      1032
           3       0.86      0.91      0.89      1010
           4       0.91      0.94      0.92       982
           5       0.88      0.86      0.87       892
           6       0.96      0.93      0.95       958
           7       0.94      0.90      0.92      1028
           8       0.91      0.84      0.87       974
           9       0.92      0.88      0.90      1009

    accuracy                           0.92     10000
   macro avg       0.92      0.91      0.91     10000
weighted avg       0.92      0.92      0.92     10000



# Cross-validation using gridsearch to tune the hyper-parameter 'C' and 'gamma'

In [63]:
from sklearn.model_selection import GridSearchCV

In [69]:
param_grid = {'C': [0.1,1, 0.01], 'gamma': [0.01, 0.1,1]}

In [70]:
grid = GridSearchCV(SVC(kernel='linear'), param_grid, verbose=1, cv=3)
grid.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  7.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 0.01], 'gamma': [0.01, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [71]:
grid_predictions = grid.predict(X_test)
grid.best_params_

{'C': 0.1, 'gamma': 0.01}

In [73]:
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       980
           1       0.96      0.99      0.97      1135
           2       0.88      0.91      0.90      1032
           3       0.86      0.91      0.89      1010
           4       0.91      0.94      0.92       982
           5       0.88      0.86      0.87       892
           6       0.96      0.93      0.95       958
           7       0.94      0.90      0.92      1028
           8       0.91      0.84      0.87       974
           9       0.92      0.88      0.90      1009

    accuracy                           0.92     10000
   macro avg       0.92      0.91      0.91     10000
weighted avg       0.92      0.92      0.92     10000



In [74]:
grid.score(X_test, y_test)

0.9153

# Question
Given (x1, y1), ...,(xn,yn) where y1, ...,yn $\in $ { -1, 1 }

## Primal problem  
$ minimize\quad w^T.w + C\sum_{i=1}^{N}\xi $  
subject to 
$ y_i.(w^T.x_i) \geq 1-\xi  ,and, \xi \geq 0 for i=1,..N $

## Lagragian

$ L(w,\xi,\alpha,\gamma) = \frac{1}{2} w^T.w + C\sum_{i=1}^{N}\xi_i - \sum_{i=1}^{N}\alpha_i[y_i(w^T.x_i) - 1 + \xi] - \sum_{i=1}^{N}\gamma_i\xi_i $  
where $ \alpha_i \geq 0 \quad 
\quad and \quad \gamma_i \geq 0 $

## Set partial derivative of Lagrange function over primal variable to zero

$ \frac{\partial L}{\partial w} = w - \sum_{i=1}^{N}y_i\alpha_ix_i = 0 $  
$ w = \sum_{i=1}^{N}y_i\alpha_i x_i $  
$ \frac{\partial L}{\partial\xi} = C - \alpha - \gamma = 0 $

Substitute $ w = \sum_{i=1}^{N}y_i\alpha_i x_i $ 
    and $ \gamma = C - \alpha $

## The dual problem
$ maximize\quad L(\alpha, \gamma) = \frac{1}{2} (\sum_{i=1}^{N}y_i\alpha_i x_i)^T.(\sum_{j=1}^{N}y_j\alpha_j x_j) + C\sum_{i=1}^{N}\xi_i - \sum_{i=1}^{N}\alpha_i[y_i((\sum_{i=1}^{N}y_i\alpha_i x_i)^T.x_i) - 1 + \xi] - \sum_{i=1}^{N}\gamma_i\xi_i $ 

$ \implies maximize\quad L(\alpha, \gamma) = \sum_{i=1}^{N}\alpha_i - \frac{1}{2} \sum_{i=1}^{N}\sum_{j=1}^{N}y_iy_j\alpha_i\alpha_jx_j^Tx_i $ 

 

subject to 
$ 0 \leq \alpha_i \leq C $ and $ \sum_{i=1}^{N}\alpha_iy_i = 0 $

## Margins
- Margin in primal formulation
$ (\sqrt{w^Tw})^\frac{-1}{2} $
- Margin in the dual formulation
$ (\sqrt{\sum_{i=1}^{N}\sum_{j=1}^{N}y_iy_j\alpha_i\alpha_jx_j^Tx_i})^\frac{-1}{2} $

## Benefit of maximizing the margin
The data that we get may have some noise and hence may not always follow the support vectors. If the margin is maximized, it reduces the affect of noise on our classifier accuracy.

## Benefit of solving a dual problem
 - The dual solution provides a lower bound on the primal solution.
 - In some cases finding the dual solution of a given primal solution is easier.
 - For eg: If the primal problem is minimization but finding the maximum of dual will provide the minimum of primal.
 
## Characterize support vectors
1. The vectors on the margin i.e. ( $\xi = 0 $)
2. Vectors on the incorrect side of the dividing hyperplane i.e. ( $ 0 \lt \xi \lt 1 $)
3. Vectors on the incorrect side of the dividing hyperplane i.e. ( $\xi \geq 1 $)